In [11]:
import pandas as pd
import numpy as np
from datetime import timedelta

from pystac_client import Client
import planetary_computer as pc

import rasterio
from rasterio.windows import from_bounds
from pyproj import Transformer
from tqdm.auto import tqdm

STAC_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
COLLECTION = "sentinel-2-l2a"

BANDS = ["B02", "B03", "B04", "B08", "B11", "B12"]  # Blue, Green, Red, NIR, SWIR1, SWIR2

In [12]:
WQ_TRAIN_PATH = "water_quality_training_dataset.csv"

df = pd.read_csv(WQ_TRAIN_PATH)
df.columns = [c.strip() for c in df.columns]

# Robust mixed-format parsing:
# 1) try dayfirst
df["Sample Date"] = pd.to_datetime(df["Sample Date"], errors="coerce", dayfirst=True)

# 2) fallback for remaining NaT (often mm-dd-YYYY)
mask = df["Sample Date"].isna()
df.loc[mask, "Sample Date"] = pd.to_datetime(df.loc[mask, "Sample Date"], errors="coerce", format="%m-%d-%Y")

# Ensure numeric coords
df["Latitude"] = pd.to_numeric(df["Latitude"], errors="coerce")
df["Longitude"] = pd.to_numeric(df["Longitude"], errors="coerce")

df = df.dropna(subset=["Latitude", "Longitude", "Sample Date"]).reset_index(drop=True)

print("Rows after cleaning:", len(df))
print(df[["Latitude","Longitude","Sample Date"]].head())

Rows after cleaning: 9319
    Latitude  Longitude Sample Date
0 -28.760833  17.730278  2011-01-02
1 -26.861111  28.884722  2011-01-03
2 -26.450000  28.085833  2011-01-03
3 -27.671111  27.236944  2011-01-03
4 -27.356667  27.286389  2011-01-03


In [16]:
from datetime import timedelta
import pandas as pd

def pick_best_s2_item(lon, lat, sample_date, days=7, cloud_lt=30):
    """
    Find Sentinel-2 L2A item near (lon,lat) around sample_date.
    Picks the lowest-cloud item, then closest in time.
    Fixes tz-naive vs tz-aware mismatch by converting everything to UTC.
    """
    # ensure tz-aware UTC
    sample_date = pd.Timestamp(sample_date)
    if sample_date.tzinfo is None:
        sample_date = sample_date.tz_localize("UTC")
    else:
        sample_date = sample_date.tz_convert("UTC")

    start = (sample_date - timedelta(days=days)).date().isoformat()
    end   = (sample_date + timedelta(days=days)).date().isoformat()

    search = catalog.search(
        collections=[COLLECTION],
        intersects={"type": "Point", "coordinates": [lon, lat]},
        datetime=f"{start}/{end}",
        query={"eo:cloud_cover": {"lt": cloud_lt}},
        limit=100
    )

    # new API: items() (no get_items)
    items = list(search.items())
    if not items:
        return None

    def key(it):
        cloud = it.properties.get("eo:cloud_cover", 9999)
        it_time = pd.Timestamp(it.datetime)
        if it_time.tzinfo is None:
            it_time = it_time.tz_localize("UTC")
        else:
            it_time = it_time.tz_convert("UTC")
        dt = abs((it_time - sample_date).total_seconds())
        return (cloud, dt)

    items.sort(key=key)
    return items[0]

In [17]:
def read_band_mean(item, band, lon, lat, buffer_m=60):
    """
    Mean reflectance (or scaled integer) in a square buffer around the point.
    buffer_m in meters (works because source CRS is usually UTM).
    """
    if band not in item.assets:
        return np.nan

    href = pc.sign(item.assets[band].href)

    with rasterio.open(href) as src:
        transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
        x, y = transformer.transform(lon, lat)

        win = from_bounds(
            x - buffer_m, y - buffer_m,
            x + buffer_m, y + buffer_m,
            transform=src.transform
        )

        arr = src.read(1, window=win, masked=True)
        if arr.size == 0:
            return np.nan

        val = float(arr.mean())

    # Sentinel-2 reflectance is commonly scaled by 10000 in many archives.
    # Heuristic: if value looks like 0..10000, scale down.
    if np.isfinite(val) and val > 2.0:
        val = val / 10000.0

    return val

In [18]:
rows = []

for i in tqdm(range(len(df))):
    lat = float(df.loc[i, "Latitude"])
    lon = float(df.loc[i, "Longitude"])
    sdate = pd.Timestamp(df.loc[i, "Sample Date"]).tz_localize("UTC")

    # Sentinel-2 L2A coverage issue (2016+ practical). Earlier dates -> no items.
    item = pick_best_s2_item(lon, lat, sdate, days=7, cloud_lt=30)

    out = {
        "Latitude": lat,
        "Longitude": lon,
        "Sample Date": sdate.date().isoformat(),
        "s2_item_id": None,
        "s2_datetime": None,
        "s2_cloud": np.nan
    }

    if item is None:
        # keep NaNs for bands
        for b in BANDS:
            out[b] = np.nan
        out["NDVI"] = np.nan
        out["NDWI"] = np.nan
        out["NDMI"] = np.nan
        rows.append(out)
        continue

    out["s2_item_id"] = item.id
    out["s2_datetime"] = pd.Timestamp(item.datetime).isoformat()
    out["s2_cloud"] = item.properties.get("eo:cloud_cover", np.nan)

    # bands
    for b in BANDS:
        out[b] = read_band_mean(item, b, lon, lat, buffer_m=60)

    # indices (optional)
    B03, B04, B08, B11 = out["B03"], out["B04"], out["B08"], out["B11"]

    out["NDVI"] = (B08 - B04) / (B08 + B04) if np.isfinite(B08) and np.isfinite(B04) and (B08 + B04) != 0 else np.nan
    out["NDWI"] = (B03 - B08) / (B03 + B08) if np.isfinite(B03) and np.isfinite(B08) and (B03 + B08) != 0 else np.nan
    out["NDMI"] = (B08 - B11) / (B08 + B11) if np.isfinite(B08) and np.isfinite(B11) and (B08 + B11) != 0 else np.nan

    rows.append(out)

feat = pd.DataFrame(rows)
feat.to_csv("sentinel2_features.csv", index=False)
print("Saved: sentinel2_features.csv")
feat.head()

100%|██████████████████████████████████████▊| 9273/9319 [56:55<00:50,  1.09s/it]/var/folders/hv/kxrxx3qn2rz5xqvmhb82grsw0000gn/T/ipykernel_5197/2097992779.py:25: UserWarning: Warning: converting a masked element to nan.
  val = float(arr.mean())
100%|███████████████████████████████████████| 9319/9319 [59:46<00:00,  2.60it/s]


Saved: sentinel2_features.csv


,Latitude,Longitude,Sample Date,s2_item_id,s2_datetime,s2_cloud,B02,B03,B04,B08,B11,B12,NDVI,NDWI,NDMI
0,-28.760833,17.730278,2011-01-02,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-26.861111,28.884722,2011-01-03,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-26.450000,28.085833,2011-01-03,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-27.671111,27.236944,2011-01-03,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-27.356667,27.286389,2011-01-03,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
